In [ ]:
import great_expectations as gx
from great_expectations.core.batch import BatchRequest
import os

# Define paths
project_root = os.path.dirname(os.path.abspath(_file_))
data_path = os.path.join(project_root, "data", "your_dataset.csv")

# Initialize a Data Context
context = gx.get_context()

# Add a Pandas datasource
datasource_name = "my_pandas_datasource"
context.sources.add_pandas_filesystem(
    name=datasource_name,
    base_directory=os.path.join(project_root, "data")
)

# Add a CSV data asset
data_asset_name = "my_csv_data_asset"
context.sources[datasource_name].add_csv_asset(
    name=data_asset_name,
    batching_regex=r"your_dataset\.csv"
)

# Create a Batch Request
batch_request = BatchRequest(
    datasource_name=datasource_name,
    data_connector_name="default_inferred_data_connector_name",
    data_asset_name=data_asset_name
)

# Create an Expectation Suite
expectation_suite_name = "my_expectation_suite"
context.add_or_update_expectation_suite(expectation_suite_name)

# Get a Validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Define Expectations
validator.expect_column_values_to_not_be_null(column="column_name")
validator.expect_column_values_to_be_unique(column="unique_column")
validator.expect_column_values_to_be_between(column="numeric_column", min_value=0, max_value=100)

# Save the Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

# Run a Checkpoint
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="my_checkpoint",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
)
checkpoint_result = checkpoint.run()

# Print Validation Results
print(checkpoint_result)